In [5]:
from lark import Lark, Transformer, v_args
from typing import Dict, Any, List, Tuple, Optional

# --- 1. GALAXY_TYPES 定义 (不变) ---
GALAXY_TYPES = [
    "void", "bool", "int", "fixed", "text", "string", "color",
    "unit", "unitgroup", "player", "playergroup", "point", "sound",
    "timer", "abilcmd", "wavestream", "rect", "bank", "dialog", "ditem",
    "struct" 
]
GALAXY_TYPES_OR_FORMAT = "(" + " | ".join(f'"{t}"' for t in GALAXY_TYPES) + ")"

# 修复后的 Lark 语法定义 (不变)
GRAMMAR = r"""
start: (declaration | comment_ignore)*

declaration: const_decl
           | var_decl
           | func_decl
           | trigger_decl

// --------------------- 类型和维度 ---------------------

base_type: """ + GALAXY_TYPES_OR_FORMAT + r"""

full_type: base_type array_dimensions?

array_dimensions: array_dimension+
array_dimension: LSQB [INT_LITERAL] RSQB

// --------------------- 函数声明 ---------------------

func_decl: base_type C_NAME LPAR [param_list] RPAR SEMICOLON
param_list: param (COMMA param)*
param: full_type C_NAME

// --------------------- 常量和变量声明 ---------------------

const_decl: CONST full_type C_NAME EQ value SEMICOLON

var_decl: full_type C_NAME SEMICOLON

// --------------------- 触发器声明 ---------------------

trigger_decl: TRIGGER C_NAME SEMICOLON

// --------------------- 命名模式和字面量 ---------------------

C_NAME: /lib[A-Z0-9]+_[gG][eEvVfTtF]_[A-Za-z0-9_]+/ | /[A-Za-z_][A-Za-z0-9_]+/

value: (INT_LITERAL | FIXED_LITERAL | QUOTED_STRING | FUNCTION_CALL | CONSTANT_NATIVE)

CONSTANT_NATIVE: /c_[A-Za-z0-9_]+/ | /e_[A-Za-z0-9_]+/

FUNCTION_CALL: /Color\s*\(\s*(?:-?\d+(?:\.\d*)?,?\s*)+\)/
QUOTED_STRING: /"[^"]*"/
INT_LITERAL: /(-?\d+)/
FIXED_LITERAL: /(-?\d+\.\d+)/

// --------------------- 词元定义 ---------------------

CONST: "const"
TRIGGER: "trigger"
SEMICOLON: ";"
EQ: "="
LPAR: "("
RPAR: ")"
LSQB: "["
RSQB: "]"
COMMA: ","

// --------------------- 忽略/辅助 ---------------------

comment_ignore: "include" QUOTED_STRING

%ignore /\s+/
%ignore /\/\/[^\n]*/
%ignore /\/\*(.|\n)*?\*\//
"""

# GalaxySymbol 类定义 (最终修复了 _resolve_name 的索引错误)
class GalaxySymbol:
    def __init__(self, full_name: str, data_type: str, is_const: bool,
                 value: Optional[str] = None, params: Optional[List[Tuple[str, str]]] = None):
        self.full_name = full_name
        self.data_type = data_type
        self.is_const = is_const
        self.value = value
        self.params = params if params is not None else []
        self.library_id, self.object_type, self.short_name = self._resolve_name(full_name)

    def _resolve_name(self, name: str) -> Tuple[str, str, str]:
        parts = name.split('_', 2)
        
        # --- 1. 处理 lib_name 模式 ---
        if parts[0].startswith("lib"):
            if len(parts) >= 3:
                return parts[0][3:], parts[1], parts[2]
            else:
                 return "", "", name 
        
        # --- 2. 处理 lp_/lv_ 等局部参数名模式 ---
        if parts[0].startswith('l') and len(parts[0]) <= 3:
            if len(parts) > 1:
                # 关键修复: 安全地获取 object_type
                object_type = parts[0][1:] if len(parts[0]) > 1 else ""
                return "", object_type, parts[1]
            else:
                return "", "", name
                
        # --- 3. 其它/未知模式 ---
        return "", "", name

    def __repr__(self):
        type_info = f"Const: {self.is_const}, TypePrefix: {self.object_type}"
        return f"<Symbol {self.full_name} | {type_info}>"

# 2. Lark Transformer (保持不变)

@v_args(inline=True)
class GalaxyScriptTransformer(Transformer):
    def __init__(self):
        super().__init__()
        self.symbol_table: Dict[str, GalaxySymbol] = {}

    def full_type(self, base_type, array_dims=None):
        if array_dims:
            return str(base_type) + "".join(str(d) for d in array_dims)
        return str(base_type)

    def base_type(self, token=None): 
         if token:
             return str(token)
         return "" 

    # Rule: array_dimension: LSQB [INT_LITERAL] RSQB
    def array_dimension(self, *children):
        if len(children) == 3:
            dim = str(children[1])
        else:
            dim = ''
            
        return f"[{dim}]"

    def param(self, full_type_string, name_token):
        return (str(full_type_string), str(name_token))

    def array_dimensions(self, *dims):
        return [str(d) for d in dims]

    # const_decl: CONST full_type C_NAME EQ value SEMICOLON
    def const_decl(self, const_token, full_type, name_token, eq_token, value_token, semicolon):
        full_name = str(name_token)
        data_type = str(full_type)
        value = str(value_token).strip()
             
        self.symbol_table[full_name] = GalaxySymbol(full_name=full_name, data_type=data_type, is_const=True, value=value)
        return name_token

    def var_decl(self, full_type, name_token, semicolon):
        full_name = str(name_token)
        data_type = str(full_type)
        self.symbol_table[full_name] = GalaxySymbol(full_name=full_name, data_type=data_type, is_const=False)
        return name_token

    # func_decl: base_type C_NAME LPAR [param_list] RPAR SEMICOLON
    def func_decl(self, base_type, name_token, lpar, *args):
        full_name = str(name_token)
        return_type = str(base_type)
        params = []
        
        param_list = None
        remaining_args = list(args)
        
        if remaining_args and isinstance(remaining_args[0], list):
             param_list = remaining_args.pop(0)

        if param_list:
            params = param_list

        self.symbol_table[full_name] = GalaxySymbol(full_name=full_name, data_type=f"function({return_type})", is_const=False, params=params)
        return name_token

    def trigger_decl(self, trigger_token, name_token, semicolon):
        full_name = str(name_token)
        self.symbol_table[full_name] = GalaxySymbol(full_name=full_name, data_type="trigger", is_const=False)
        return name_token

    # --- 忽略/辅助/终结符 ---
    def param_list(self, *params): return [p for p in params if p is not None]
    def comment_ignore(self, *args): return None
    def C_NAME(self, token): return str(token)
    
    def value(self, *children): 
        if children:
            return str(children[0])
        return ""
    
    def INT_LITERAL(self, token): return str(token)
    def FIXED_LITERAL(self, token): return str(token)
    def QUOTED_STRING(self, token): return str(token)
    def FUNCTION_CALL(self, token): return str(token)
    def CONSTANT_NATIVE(self, token): return str(token)


def parse_galaxy_script_header(code: str) -> Dict[str, GalaxySymbol]:
    """
    使用 Lark 解析器解析 Galaxy Script 头文件代码并返回符号表。
    """
    parser = Lark(GRAMMAR, parser='earley', propagate_positions=True, maybe_placeholders=False)
    tree = parser.parse(code)
    transformer = GalaxyScriptTransformer()
    transformer.transform(tree)
    return transformer.symbol_table

# --- 3. 示例使用 ---

header_code = """
include "TriggerLibs/NativeLib"

// 常量
const int libEA0820A1_ge_UnitSelectionStoreOption_ClearUnitSelection = 0;
const fixed libEA0820A1_gv_campaignObjectiveDelay = 2.0;
const color libEA0820A1_gv__CBC_LINE_DEFAULT_TEXTCOLOR = Color(0,0,0);
const int libEA0820A1_ge_MissionObjective_TRaynor01Objective1 = 1;
const int libEA0820A1_ge_MapID_MapTSecret01 = c_triggerControlTypeImage;

// 变量声明
point libEA0820A1_gv__TP_DropPod_Location;
unitgroup libEA0820A1_gv__TP_DropPod_Units;
text[501] libEA0820A1_gv__CB_Line;
fixed[101][17] libEA0820A1_gv__CB_Unit_ActivationDistance;
unit[33] libEA0820A1_gv__CB_Current_ChatBubbleTarget;
// 允许空尺寸数组
text[] libEA0820A1_gv_tS_MapSubtitle;

// 函数声明
void libEA0820A1_gf_DisplayCampaignMessage (playergroup lp_toPlayerGroup, int lp_messageType, text lp_message, int lp_formatOrNot);
text libEA0820A1_gf_FormatTimeCampaign (int lp_second);
void libEA0820A1_gf_SetDialogItemSizeInGrid (int lp_dialogItem, playergroup lp_players, fixed lp_width, fixed lp_height);
int libEA0820A1_gf_LastAddedChatBubble ();

// 触发器声明
trigger libEA0820A1_gt__ZergDropPod;
trigger libEA0820A1_gt_TS_AbortMission;
"""

print("--- 符号表解析结果 ---")
import sys
from io import StringIO
old_stdout = sys.stdout
redirected_output = StringIO()
sys.stdout = redirected_output

try:
    symbol_table = parse_galaxy_script_header(header_code)

    for name, symbol in symbol_table.items():
        params_str = f"({', '.join(f'{t[0]} {t[1]}' for t in symbol.params)})" if symbol.params else "()"
        print(f"[{symbol.object_type.upper()}] {symbol.data_type.ljust(30)} {symbol.full_name.ljust(50)} | Value: {symbol.value or ''} | Params: {params_str if 'function' in symbol.data_type else 'N/A'}")

    print("\n--- 详细符号信息 ---")
    print(symbol_table['libEA0820A1_ge_UnitSelectionStoreOption_ClearUnitSelection'])
    print(symbol_table['libEA0820A1_gv__CB_Line'])
    print(symbol_table['libEA0820A1_gv__CB_Unit_ActivationDistance'])
    print(symbol_table['libEA0820A1_gf_SetDialogItemSizeInGrid'])

except Exception as e:
    sys.stdout = old_stdout
    print("\n--- 解析失败 ---")
    print(f"错误类型: {type(e).__name__}")
    print(f"错误信息: {e}")
    
sys.stdout = old_stdout
print(redirected_output.getvalue())

--- 符号表解析结果 ---

--- 解析失败 ---
错误类型: IndexError
错误信息: string index out of range
[GE]                                libEA0820A1_ge_UnitSelectionStoreOption_ClearUnitSelection | Value: 0 | Params: N/A
[GV]                                libEA0820A1_gv_campaignObjectiveDelay              | Value: 2.0 | Params: N/A
[GV]                                libEA0820A1_gv__CBC_LINE_DEFAULT_TEXTCOLOR         | Value: Color(0,0,0) | Params: N/A
[GE]                                libEA0820A1_ge_MissionObjective_TRaynor01Objective1 | Value: 1 | Params: N/A
[GE]                                libEA0820A1_ge_MapID_MapTSecret01                  | Value: c_triggerControlTypeImage | Params: N/A
[GV]                                libEA0820A1_gv__TP_DropPod_Location                | Value:  | Params: N/A
[GV]                                libEA0820A1_gv__TP_DropPod_Units                   | Value:  | Params: N/A
[GV] [501]                          libEA0820A1_gv__CB_Line                            | Valu

In [7]:
symbol_table

{'libEA0820A1_ge_UnitSelectionStoreOption_ClearUnitSelection': <Symbol libEA0820A1_ge_UnitSelectionStoreOption_ClearUnitSelection | Const: True, TypePrefix: ge>,
 'libEA0820A1_gv_campaignObjectiveDelay': <Symbol libEA0820A1_gv_campaignObjectiveDelay | Const: True, TypePrefix: gv>,
 'libEA0820A1_gv__CBC_LINE_DEFAULT_TEXTCOLOR': <Symbol libEA0820A1_gv__CBC_LINE_DEFAULT_TEXTCOLOR | Const: True, TypePrefix: gv>,
 'libEA0820A1_ge_MissionObjective_TRaynor01Objective1': <Symbol libEA0820A1_ge_MissionObjective_TRaynor01Objective1 | Const: True, TypePrefix: ge>,
 'libEA0820A1_ge_MapID_MapTSecret01': <Symbol libEA0820A1_ge_MapID_MapTSecret01 | Const: True, TypePrefix: ge>,
 'libEA0820A1_gv__TP_DropPod_Location': <Symbol libEA0820A1_gv__TP_DropPod_Location | Const: False, TypePrefix: gv>,
 'libEA0820A1_gv__TP_DropPod_Units': <Symbol libEA0820A1_gv__TP_DropPod_Units | Const: False, TypePrefix: gv>,
 'libEA0820A1_gv__CB_Line': <Symbol libEA0820A1_gv__CB_Line | Const: False, TypePrefix: gv>,
 'libEA